In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import IPython.display as disp

import geemap
import requests

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project="env-16-remote-sensing")

In [ ]:
imageCollectionInfoUrl = 'https://storage.googleapis.com/earthengine-stac/catalog/COPERNICUS/COPERNICUS_S2_SR_HARMONIZED.json'
r = requests.get(imageCollectionInfoUrl)
imageCollInfo = r.json()
print(imageCollInfo)

In [ ]:
bands = []

for key, value in imageCollInfo.items():
    if key == 'summaries':
        for key2, value2 in value.items():
            if key2 == 'eo:bands':
                for band in value2:
                    print(f'{band["name"]}: {band}')
                    bands.append(band['name'])

In [ ]:
geometry = ee.Geometry.Point([9.269243, 45.4654219])
roi = geometry.buffer(500000)


collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
                .filterBounds(geometry) \
                .filterDate('2022-01-01', '2023-04-10') \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)) \
                .sort("CLOUDY_PIXEL_PERCENTAGE") \
                .select(bands)


print(collection.size().getInfo())


In [ ]:
collection.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()


In [ ]:
imageList = collection.toList(collection.size())

image = imageList.get(0)
image = ee.Image(image)

In [ ]:
Map = geemap.Map()
vis_param = {'min': 0,
             'max': 3000,
            #  'bands': ['B4', 'B3', 'B2'],
            'bands': ['B11', 'B12', 'B2'],
             'gamma': 1.5}

Map.addLayer(image, vis_param, "First image")
Map.centerObject(geometry, 13)

Map

In [ ]:
# gdexport = ee.batch.Export.image.toDrive(image.clip(roi),
#                                          description='driveExportTask_s2',
#                                          folder = 'gee',
#                                          fileNamePrefix = 's2_Milan_test_big',
#                                          scale = 10,
#                                          maxPixels = 1e11)
# gdexport.start()

In [ ]:
# gdexport.status()

In [ ]:
run scripts/dispms -f data/s2_Milan_test_big.tif -e 4 -p [1,2,3] -d [600,600,3000,3000]

In [ ]:
run scripts/S2cnnclassify -d [600,600,3000,3000] -s 8 data/s2_Milan_test_big.tif